In [1]:
import pandas as pd
import numpy as np 
import xgboost as xgb
from collections import Counter
import re
import sklearn.cross_validation as cv

In [2]:
def Gini(y_pred, dtrain):
    y_true = dtrain.get_label()
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    
    # sort rows on prediction column 
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:,0].argsort()][::-1,0]
    pred_order = arr[arr[:,1].argsort()][::-1,0]
    
    # get Lorenz curves
    L_true = np.cumsum(true_order) / np.sum(true_order)
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    L_ones = np.linspace(0, 1, n_samples)
    
    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)
    
    # normalize to true Gini coefficient
    return 'Gini', -G_pred/G_true

## Train

In [ ]:
import warnings
warnings.filterwarnings('ignore')
num_rounds = 10000
params = {  "objective": "reg:linear"
          , "eta": 0.003
          , "min_child_weight": 50
          , "subsample": 0.6
          , "colsample_bytree": 0.7
          , "scale_pos_weight": 1
          , "silent": 1
          , "max_depth": 9}

digs = re.compile(r'\d+')

cat_to_drop = []

tr = pd.read_csv('./train.csv', sep = ',', index_col = 'Id')
te =  pd.read_csv('./test.csv', sep = ',',index_col = 'Id')

labels = tr['Hazard'].values

for i in range(32):
    full =  pd.concat(objs = [tr,te])
    full.drop(cat_to_drop, axis=1, inplace = 1)
    full_cat = full.select_dtypes(include = ['object'])
    full_num = full.select_dtypes(exclude = ['object'])
    full_cat = full_cat.apply(func = lambda x: np.unique(x, return_inverse =1)[1], axis=0)

    full = pd.concat([full_num,full_cat], axis=1)
    split = np.isnan(full.Hazard)

    train = full[~split].drop('Hazard', axis=1).values
    feature_names = full.columns[1:]
    gini = []
    for j in range(3):
        folds = cv.StratifiedKFold(labels, n_folds = 5, shuffle=True)
        for train_ind, val_ind in folds: 
            xgtrain = xgb.DMatrix(train[train_ind,:], label=labels[train_ind])
            xgval = xgb.DMatrix(train[val_ind,:], label=labels[val_ind])
            watchlist = [(xgtrain, 'train'),(xgval, 'val')]
            model = xgb.train(params, xgtrain, num_rounds, watchlist,
                              feval= Gini, early_stopping_rounds=120,verbose_eval=False)
            gini.append(model.best_score)
    importance = Counter(model.get_fscore()).most_common()
    least_imp_id = re.findall(digs, importance[-1][0])
    least_imp_name = feature_names[least_imp_id]
    print('CV repeats: {} | Mean Gini: {:.6f} | Std Gini: {:.6f} | Dropped: {} | Least Important: {}'.format(
            len(gini), np.mean(gini),np.std(gini), cat_to_drop, least_imp_name))
    cat_to_drop.append(least_imp_name)

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1799]	train-Gini:-0.556876	val-Gini:-0.384338

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1841]	train-Gini:-0.555698	val-Gini:-0.392915

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[2330]	train-Gini:-0.579262	val-Gini:-0.379900

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1743]	train-Gini:-0.558026	val-Gini:-0.388484

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1661]	train-Gini:-0.550648	val-Gini:-0.394420

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1771]	train-Gini:-0.553665	val-Gini:-0.387965

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1532]	train-Gini:-0.543627	val-Gini:-0.392364

Will train until val error hasn't decreased in 120 rounds.
Stopping. 

CV repeats: 32 | Mean Gini: -0.387706 | Std Gini: 0.005747 | Dropped: [] | Least Important: T2_V8
CV repeats: 31 | Mean Gini: -0.386787 | Std Gini: 0.009282 | Dropped: ['T2_V8'] | Least Important: T2_V12

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[2475]	train-Gini:-0.583882	val-Gini:-0.383894

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[2186]	train-Gini:-0.572751	val-Gini:-0.407203

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[2078]	train-Gini:-0.574541	val-Gini:-0.386959

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1975]	train-Gini:-0.572260	val-Gini:-0.368737

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1690]	train-Gini:-0.549491	val-Gini:-0.389128

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1939]	train-Gini:-0.568776	val-Gini:-0.381392

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1640]	train-Gini:-0.552033	val-Gini:-0.375967

Will train until val error hasn't decreased in 120 rounds.
Stopping. 


CV repeats: 30 | Mean Gini: -0.386746 | Std Gini: 0.009555 | Dropped: ['T2_V8', 'T2_V12'] | Least Important: T2_V11

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1196]	train-Gini:-0.528201	val-Gini:-0.388822

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[2018]	train-Gini:-0.567859	val-Gini:-0.382466

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[2060]	train-Gini:-0.569299	val-Gini:-0.398899

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1912]	train-Gini:-0.559671	val-Gini:-0.388958

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1960]	train-Gini:-0.561235	val-Gini:-0.376313

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[2218]	train-Gini:-0.578978	val-Gini:-0.365177

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[2028]	train-Gini:-0.563638	val-Gini:-0.401360

Will train until val error hasn't decreased in 120 rounds.
Stopping. 

...

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1743]	train-Gini:-0.555155	val-Gini:-0.390050

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1188]	train-Gini:-0.526369	val-Gini:-0.377300

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1830]	train-Gini:-0.555245	val-Gini:-0.387791

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[2111]	train-Gini:-0.568864	val-Gini:-0.376950

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[2291]	train-Gini:-0.576311	val-Gini:-0.385206

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[2147]	train-Gini:-0.573064	val-Gini:-0.387870

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1452]	train-Gini:-0.536306	val-Gini:-0.386914

Will train until val error hasn't decreased in 120 rounds.
Stopping. 


CV repeats: 27 | Mean Gini: -0.383912 | Std Gini: 0.005497 | Dropped: ['T2_V8', 'T2_V12', 'T2_V11', 'T1_V7', 'T1_V17'] | Least Important: T2_V3

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1898]	train-Gini:-0.561210	val-Gini:-0.391378

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[2069]	train-Gini:-0.567892	val-Gini:-0.387448

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1801]	train-Gini:-0.555298	val-Gini:-0.382380

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1865]	train-Gini:-0.559351	val-Gini:-0.378368

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1605]	train-Gini:-0.544637	val-Gini:-0.388075

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1397]	train-Gini:-0.533678	val-Gini:-0.383935

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1607]	train-Gini:-0.544757	val-Gini:-0.390815

Will train until val error hasn't decreased in 120 rounds.
Stopping. 


CV repeats: 26 | Mean Gini: -0.384583 | Std Gini: 0.009558 | Dropped: ['T2_V8', 'T2_V12', 'T2_V11', 'T1_V7', 'T1_V17', 'T2_V3'] | Least Important: T1_V9

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1572]	train-Gini:-0.539403	val-Gini:-0.386944

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1892]	train-Gini:-0.558578	val-Gini:-0.390474

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1645]	train-Gini:-0.548259	val-Gini:-0.380494

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1524]	train-Gini:-0.542221	val-Gini:-0.389743

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1565]	train-Gini:-0.548704	val-Gini:-0.379049

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1428]	train-Gini:-0.540617	val-Gini:-0.387131

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[1478]	train-Gini:-0.542847	val-Gini:-0.385786

Will train until val error hasn't decreased in 120 rounds.
Stopping. 

In [3]:
import warnings
warnings.filterwarnings('ignore')
num_rounds = 10000
params = {  "objective": "reg:linear"
          , "eta": 0.01
          , "min_child_weight": 10
          , "subsample": 0.6
          , "colsample_bytree": 0.7
          , "scale_pos_weight": 1
          , "silent": 1
          , "max_depth": 9}

digs = re.compile(r'\d+')

cat_to_drop = []

tr = pd.read_csv('./train.csv', sep = ',', index_col = 'Id')
te =  pd.read_csv('./test.csv', sep = ',',index_col = 'Id')

labels = tr['Hazard'].values

for i in range(32):
    full =  pd.concat(objs = [tr,te])
    full.drop(cat_to_drop, axis=1, inplace = 1)
    full_cat = full.select_dtypes(include = ['object'])
    full_num = full.select_dtypes(exclude = ['object'])
    full_cat = full_cat.apply(func = lambda x: np.unique(x, return_inverse =1)[1], axis=0)

    full = pd.concat([full_num,full_cat], axis=1)
    split = np.isnan(full.Hazard)

    train = full[~split].drop('Hazard', axis=1).values
    feature_names = full.columns[1:]
    gini = []
    for j in range(2):
        folds = cv.StratifiedKFold(labels, n_folds = 5, shuffle=True)
        for train_ind, val_ind in folds: 
            xgtrain = xgb.DMatrix(train[train_ind,:], label=labels[train_ind])
            xgval = xgb.DMatrix(train[val_ind,:], label=labels[val_ind])
            watchlist = [(xgtrain, 'train'),(xgval, 'val')]
            model = xgb.train(params, xgtrain, num_rounds, watchlist,
                              feval= Gini, early_stopping_rounds=120,verbose_eval=False)
            gini.append(model.best_score)
    importance = Counter(model.get_fscore()).most_common()
    least_imp_id = re.findall(digs, importance[-1][0])
    least_imp_name = feature_names[least_imp_id]
    print('CV repeats: {} | Mean Gini: {:.6f} | Std Gini: {:.6f} | Dropped: {} | Least Important: {}'.format(
            len(gini), np.mean(gini),np.std(gini), cat_to_drop, least_imp_name))
    cat_to_drop.append(least_imp_name)

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[581]	train-Gini:-0.652032	val-Gini:-0.389988

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[638]	train-Gini:-0.661930	val-Gini:-0.377614

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[548]	train-Gini:-0.646044	val-Gini:-0.385451

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[367]	train-Gini:-0.607911	val-Gini:-0.391818

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[588]	train-Gini:-0.649762	val-Gini:-0.368184

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[456]	train-Gini:-0.627808	val-Gini:-0.391521

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[442]	train-Gini:-0.621696	val-Gini:-0.365320

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best it

CV repeats: 10 | Mean Gini: -0.383024 | Std Gini: 0.009024 | Dropped: [] | Least Important: T2_V8
CV repeats: 10 | Mean Gini: -0.383773 | Std Gini: 0.007696 | Dropped: ['T2_V8'] | Least Important: T2_V12

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[439]	train-Gini:-0.620237	val-Gini:-0.378576

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[420]	train-Gini:-0.620443	val-Gini:-0.386652

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[690]	train-Gini:-0.671729	val-Gini:-0.380858

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[474]	train-Gini:-0.629384	val-Gini:-0.374740

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[568]	train-Gini:-0.655584	val-Gini:-0.392163

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[714]	train-Gini:-0.677630	val-Gini:-0.375008

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[432]	train-Gini:-0.624524	val-Gini:-0.381263

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best it


CV repeats: 10 | Mean Gini: -0.383287 | Std Gini: 0.006816 | Dropped: ['T2_V8', 'T2_V12'] | Least Important: T1_V17

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[327]	train-Gini:-0.590714	val-Gini:-0.390458

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[511]	train-Gini:-0.636272	val-Gini:-0.372878

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[539]	train-Gini:-0.645019	val-Gini:-0.390463

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[560]	train-Gini:-0.643113	val-Gini:-0.390618

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[454]	train-Gini:-0.627043	val-Gini:-0.361325

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[547]	train-Gini:-0.643904	val-Gini:-0.386107

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[466]	train-Gini:-0.628558	val-Gini:-0.379276

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best it

...

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[250]	train-Gini:-0.552467	val-Gini:-0.322280

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[440]	train-Gini:-0.600930	val-Gini:-0.323036

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[361]	train-Gini:-0.581197	val-Gini:-0.334845

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[368]	train-Gini:-0.588064	val-Gini:-0.321585

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[423]	train-Gini:-0.600406	val-Gini:-0.312312

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[182]	train-Gini:-0.534369	val-Gini:-0.332375

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[445]	train-Gini:-0.603179	val-Gini:-0.316539

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best it


CV repeats: 10 | Mean Gini: -0.322728 | Std Gini: 0.008545 | Dropped: ['T2_V8', 'T2_V12', 'T1_V17', 'T2_V11', 'T1_V7', 'T2_V3', 'T1_V9', 'T1_V8', 'T1_V6', 'T1_V12', 'T1_V15', 'T1_V14', 'T2_V6', 'T2_V5', 'T1_V13', 'T1_V4', 'T1_V11', 'T1_V5', 'T2_V14', 'T1_V10'] | Least Important: T2_V13

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[267]	train-Gini:-0.551050	val-Gini:-0.334315

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[221]	train-Gini:-0.533801	val-Gini:-0.304298

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[409]	train-Gini:-0.589227	val-Gini:-0.332371

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[167]	train-Gini:-0.523481	val-Gini:-0.313997

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[249]	train-Gini:-0.545621	val-Gini:-0.315962

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[282]	train-Gini:-0.555778	val-Gini:-0.322127

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best iteration:
[380]	train-Gini:-0.579092	val-Gini:-0.322640

Will train until val error hasn't decreased in 120 rounds.
Stopping. Best it


CV repeats: 10 | Mean Gini: -0.318495 | Std Gini: 0.009122 | Dropped: ['T2_V8', 'T2_V12', 'T1_V17', 'T2_V11', 'T1_V7', 'T2_V3', 'T1_V9', 'T1_V8', 'T1_V6', 'T1_V12', 'T1_V15', 'T1_V14', 'T2_V6', 'T2_V5', 'T1_V13', 'T1_V4', 'T1_V11', 'T1_V5', 'T2_V14', 'T1_V10', 'T2_V13'] | Least Important: T1_V16


XGBoostError: b'There are NAN in the matrix, however, you did not set missing=NAN'